# BUGGY for subject 4 at least. verify and then make it for all subjects.

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
def load_imu_data(file_path):
    imu_data = pd.read_pickle(file_path)
    imu_data['t'] = pd.to_datetime(imu_data['t'])
    return imu_data

In [8]:
def extract_sync_times(json_data, subject_id):
    subject_sync_info = next(item for item in json_data['sync_info'] if item['subject_id'] == subject_id)
    imu_start_time = datetime.strptime(subject_sync_info['imu_start_time'], "%H:%M:%S.%f")
    imu_clap_time = datetime.strptime(subject_sync_info['imu_clap_time'], "%H:%M:%S.%f")
    return imu_start_time, imu_clap_time

In [4]:
def align_clap_time(imu_data, imu_clap_time):
    imu_clap_dt_corrected = imu_data['t'].dt.normalize().iloc[0] + (
        imu_clap_time - imu_clap_time.replace(hour=0, minute=0, second=0, microsecond=0)
    )
    return imu_clap_dt_corrected

In [6]:
def calculate_durations(imu_data, imu_clap_dt_corrected):
    imu_start_actual = imu_data['t'].iloc[0]
    imu_end_actual = imu_data['t'].iloc[-1]
    
    if imu_start_actual <= imu_clap_dt_corrected <= imu_end_actual:
        imu_data_after_clap = imu_data[imu_data['t'] >= imu_clap_dt_corrected]
        eating_duration = (imu_data_after_clap['t'].iloc[-1] - imu_clap_dt_corrected).total_seconds()
        total_duration = (imu_end_actual - imu_start_actual).total_seconds()
        eating_percentage = (eating_duration / total_duration) * 100
    else:
        eating_duration = 0
        total_duration = (imu_end_actual - imu_start_actual).total_seconds()
        eating_percentage = 0
    
    return eating_duration, total_duration, eating_percentage


In [19]:
def process_all_subjects(json_file_path, subject_ids):
    results = []
    total_eating_duration = 0
    total_duration = 0
    
    # Load JSON data
    with open(json_file_path, 'r') as f:
        json_data = pd.read_json(f)
    
    # Loop through each subject
    for subject_id in subject_ids:
        imu_file_path = f'../data/processed/{subject_id}/{subject_id}.pkl'
        imu_data = load_imu_data(imu_file_path)
        imu_start_time, imu_clap_time = extract_sync_times(json_data, subject_id)
        imu_clap_dt_corrected = align_clap_time(imu_data, imu_clap_time)
        eating_duration, duration, eating_percentage = calculate_durations(imu_data, imu_clap_dt_corrected)
        
        # Accumulate results
        results.append({
            "Subject ID": subject_id,
            "Eating Duration (seconds)": eating_duration,
            "Total Duration (seconds)": duration,
            "Percentage of Eating Duration": eating_percentage
        })
        
        total_eating_duration += eating_duration
        total_duration += duration
    
    # Calculate overall percentage
    overall_percentage = (total_eating_duration / total_duration) * 100 if total_duration > 0 else 0
    overall_results = {
        "Total Eating Duration (seconds)": total_eating_duration,
        "Total Duration (seconds)": total_duration,
        "Overall Percentage of Eating Duration": overall_percentage
    }
    
    return results, overall_results

In [20]:
subject_ids = ['1_meal_2', '4', '5', '6', '9', '14', '17_meal_2', '18', '19', '20']
json_file_path = 'time_sync.json'
subject_results, overall_results = process_all_subjects(json_file_path, subject_ids)

# Print Results
print("Subject-wise Results:")
for result in subject_results:
    print(result)

print("\nOverall Results:")
print(overall_results)

Subject-wise Results:
{'Subject ID': '1_meal_2', 'Eating Duration (seconds)': 399.56, 'Total Duration (seconds)': 1043.04, 'Percentage of Eating Duration': 38.30725571406657}
{'Subject ID': '4', 'Eating Duration (seconds)': 0.014999, 'Total Duration (seconds)': 2792.63, 'Percentage of Eating Duration': 0.0005370922750239022}
{'Subject ID': '5', 'Eating Duration (seconds)': 931.83, 'Total Duration (seconds)': 2543.16, 'Percentage of Eating Duration': 36.640636059076115}
{'Subject ID': '6', 'Eating Duration (seconds)': 290.54, 'Total Duration (seconds)': 1588.5, 'Percentage of Eating Duration': 18.290210890777463}
{'Subject ID': '9', 'Eating Duration (seconds)': 378.67, 'Total Duration (seconds)': 451.0, 'Percentage of Eating Duration': 83.96230598669624}
{'Subject ID': '14', 'Eating Duration (seconds)': 489.13, 'Total Duration (seconds)': 1145.76, 'Percentage of Eating Duration': 42.69044127915096}
{'Subject ID': '17_meal_2', 'Eating Duration (seconds)': 506.94, 'Total Duration (seconds

In [10]:
imu_start_time

datetime.datetime(1900, 1, 1, 16, 13, 0, 902000)

In [11]:
imu_clap_time

datetime.datetime(1900, 1, 1, 16, 23, 44, 382000)

In [12]:
imu_clap_dt_corrected

Timestamp('2023-11-20 16:23:44.382000')

In [13]:
eating_duration

399.56

In [14]:
total_duration

1043.04

In [15]:
eating_percentage

38.30725571406657